# **Limpieza**

#### LIBRERÍAS

In [ ]:
import pandas as pd # Manipulación df
import os # Directorios
import re # Manipulación de cadenas de texto (expresiones regulares)
import unicodedata # Función para eliminar acentos
from datetime import datetime, date  # Para trabajar con fechas y horas
from currency_converter import CurrencyConverter # convertidor de monedas
from dateutil.relativedelta import relativedelta  # Para calcular diferencias entre fechas, incluyendo años, meses y días

#### CARGAR ARCHIVO

Solo cambia el nombre en 'archivo_filename' y su ruta con 'archivo_dir'

In [ ]:
directorio_actual = os.getcwd() # Directorio actual de trabajo
# Ruta relativa al archivo CSV
archivo_filename = "ruta/relativa/al/archivo.csv"
archivo_dir = os.path.join(directorio_actual)
archivo_path = os.path.join(archivo_dir, archivo_filename)

Elimina el # de la primer línea si el archivo a leer no contiene el nombre de las columnas

In [ ]:
#column_names=['precio','denominacion','propiedad','metros_total','metros_construido','tiempo_de_publicacion','tipo','estacionamientos','recamaras','banos','medio_banos','seguridad_privada','ubicacion','url','descripcion']
df = pd.read_csv(archivo_path)
df.info()

# Elegir tipo de archivo a leer
#df = pd.read_excel(csv_path) 
df = pd.read_csv(archivo_path)

#### LIMPIEZA NULOS

In [ ]:
df.fillna({'precio': 0}, inplace=True) 
#df.fillna({'denominacion': 'MXN'}, inplace=True) 
df.fillna({'propiedad': 'Propiedad sin título'}, inplace=True) 
df.fillna({'metros_total': 0}, inplace=True) 
df.fillna({'metros_construido': 0}, inplace=True) 
#df.fillna({'tiempo_de_publicacion': '1 ene 1000'}, inplace=True) 
df.fillna({'tipo': 'Propiedad sin tipo de casa'}, inplace=True) 
df.fillna({'estacionamientos': 0}, inplace=True) 
df.fillna({'recamaras': 0}, inplace=True) 
df.fillna({'banos': 0}, inplace=True) 
df.fillna({'medio_banos': 0}, inplace=True) 
df.fillna({'seguridad_privada': 'No'}, inplace=True) 
df.fillna({'ubicacion': 'Propiedad sin ubicación'}, inplace=True) 
df.fillna({'url': '-'}, inplace=True) 
df.fillna({'descripcion': "Propiedad sin descripcion"}, inplace=True) 

# 1. Manipulación de precios

## a) Precio en descripción

Agrega las diferentes formas de que el precio se encuentre en la descripción.

In [ ]:
def extraer_precio(descripcion):
    descripcion = descripcion.lower()
    # Buscar patrones
    precios = re.findall(r'\b(\d+[\.,]?\d*[\.,]?\d*)\s*($|mxn|USD|precio|pesos|monto|)', descripcion)
    precio = None
    for match in precios:
        numero, moneda = match
        numero = numero.replace(',', '').replace("'", '').replace(".",'')
        try:
            precio = float(numero)
        except ValueError:
            continue
        if precio:
            break
    return precio

# Denominación de la descripción
def extraer_denominacion(descripcion):
    descripcion = descripcion.lower()
    # Buscar patrones 
    denominaciones = re.findall(r'\b(mil|mxn|precio|pesos|monto|Mexicanos|Estadounidenses)\b', descripcion)
    return denominaciones[0] if denominaciones else None

df['precio_desc'] = df['descripcion'].apply(extraer_precio)
#df['denominacion_desc'] = df['descripcion'].apply(extraer_denominacion)

### Precio [0 / 0]

In [ ]:
# Filtrar los registros según las condiciones dadas
totales = df[( (df['precio'] == 0 ) & (df['precio_desc'].isna()) )] 
pd.set_option('display.max_colwidth', None) # Mostrar URLs completas
# Iterar sobre los registros filtrados
for idx, row in totales.iterrows():
    print(f"Registro {idx}:")
    print(row[['precio', 'precio_desc', 'url']])
    print("\n¿Qué acción te gustaría realizar?")
    print("1. Sustituir el valor manualmente")
    print("2. Pasar al siguiente registro")

    # Solicitar la opción al usuario
    option = input("Elige una opción (1 o 2): ")

    if option == '1':
        # Sustituir el valor manualmente
        new_value = input("Introduce el nuevo valor para 'precio': ")
        df.at[idx, 'precio'] = float(new_value)
    elif option == '2':
        # Pasar al siguiente registro
        continue
    else:
        print("Opción no válida. Pasando al siguiente registro...")
    
    print("\n---\n")

print("Proceso completado.")

Proceso completado.


Eliminar registros que no recuperaron su precio ni desde la descripción

In [ ]:
df = df[~( (df['precio'] == 0 ) & (df['precio_desc'].isna()) )] 

### Precio [0 / #]

In [ ]:
# Filtrar los registros según las condiciones dadas
totales = df[( (df['precio'] == 0 ) & (df['precio_desc'].notna()) )] 
pd.set_option('display.max_colwidth', None) # Mostrar URLs completas

# Iterar sobre los registros filtrados
for idx, row in totales.iterrows():
    print(f"Registro {idx}:")
    print(row[['precio', 'precio_desc', 'url']])
    print("\n¿Qué acción te gustaría realizar?")
    print("1. Sustituir el valor manualmente")
    print("2. Sustituir el valor de 'precio' por el de 'precio_desc'")
    print("3. Pasar al siguiente registro")

    # Solicitar la opción al usuario
    option = input("Elige una opción (1, 2, 3): ")

    if option == '1':
        # Sustituir el valor manualmente
        new_value = input("Introduce el nuevo valor para 'precio': ")
        df.at[idx, 'precio'] = float(new_value)
    elif option == '2':
        # Sustituir el valor de 'metros_total' por 'totales_desc'
        df.at[idx, 'precio'] = df.at[idx, 'precio_desc']
    elif option == '3':
        # Pasar al siguiente registro
        continue
    else:
        print("Opción no válida. Pasando al siguiente registro...")
    
    print("\n---\n")

print("Proceso completado.")


Proceso completado.


### Precio [# / #]

In [ ]:
# Filtrar los registros según las condiciones dadas
totales = df[( ((df['precio']!= 0) & (df['precio_desc'].notna())) & (df['precio'] != df['precio_desc']) )] 
# Filtro dentro del estándar
totales = totales[(totales['precio_desc'] >= 250000) & (totales['precio_desc'] <= 500000000)]
pd.set_option('display.max_colwidth', None) # Mostrar URLs completas

# Iterar sobre los registros filtrados
for idx, row in totales.iterrows():
    print(f"Registro {idx}:")
    print(row[['precio', 'precio_desc', 'url']])
    print("\n¿Qué acción te gustaría realizar?")
    print("1. Sustituir el valor manualmente")
    print("2. Sustituir el valor de 'precio' por el de 'precio_desc'")
    print("3. Pasar al siguiente registro")

    # Solicitar la opción al usuario
    option = input("Elige una opción (1, 2, 3): ")

    if option == '1':
        # Sustituir el valor manualmente
        new_value = input("Introduce el nuevo valor para 'precio': ")
        df.at[idx, 'precio'] = float(new_value)
    elif option == '2':
        # Sustituir el valor de 'metros_total' por 'totales_desc'
        df.at[idx, 'precio'] = df.at[idx, 'precio_desc']
    elif option == '3':
        # Pasar al siguiente registro
        continue
    else:
        print("Opción no válida. Pasando al siguiente registro...")

    print("\n---\n")

print("Proceso completado.")

## b) Denominación MXN

SI EXISTE UNA DENOMINACIÓN QUE NO SEA MXN, SE CONVIERTA EL PRECIO

In [ ]:
import datetime
c = CurrencyConverter() # Instancia del convertidor de monedas
def convertir_a_mxn(precio, denominacion):
    if denominacion != 'MXN':
        return int(round(c.convert(precio, denominacion, 'MXN')))
    return precio

def convertir_a_usd(precio, denominacion):
    if denominacion != 'USD':
        return round(c.convert(precio, denominacion, 'USD'))
    return precio

# Obtener la fecha actual
fecha_conversion = datetime.datetime.now().date()

# Aplicar la conversión a cada fila del DataFrame y agregar la fecha de conversión
df['precio_mxn'] = df.apply(lambda row: convertir_a_mxn(row['precio'], row['denominacion']), axis=1)
df['precio_usd'] = df.apply(lambda row: convertir_a_usd(row['precio'], row['denominacion']), axis=1)
df['fecha_conversion'] = fecha_conversion
print(df[['precio', 'denominacion', 'precio_mxn', 'precio_usd', 'fecha_conversion']].head())

    precio denominacion  precio_mxn  precio_usd fecha_conversion
0  8000000          MXN     8000000   444999.64       2024-09-19
1  4810000          MXN     4810000   267556.03       2024-09-19
2  4500000          MXN     4500000   250312.30       2024-09-19
3  4300000          MXN     4300000   239187.31       2024-09-19
4  3946465          MXN     3946465   219521.94       2024-09-19


## c) Precio fuera del estandar

IDENTIFICAR Y ELIMINAR PROPIEDADES QUE CUENTEN CON PRECIOS BAJOS y ALTOS AL ESTANDAR

In [ ]:
# Eliminar los registros donde el precio es menor al estandar (250,000) y con precio_desc vacío
df = df[(df['precio'] >= 250000)] #& (df['precio_desc'].notna())]
# Eliminar los registros donde el precio es mayor al estandar (500,000,000) y con precio_desc vacío
df = df[(df['precio'] <= 500000000)] #& (df['precio_desc'].notna())] 
# Ordenar valores de manera ascendente
df['precio'].sort_values()

1605      415000
1604      419710
1603      419770
1598      419770
1602      419770
          ...   
4       24000000
3       24000000
2       26000000
1       27000000
0       32000000
Name: precio, Length: 1607, dtype: int64

## D) Segmento o categoría

In [ ]:
# Crear un diccionario donde las llaves son las categorías y los valores son las tuplas de rangos
rangos_precio = {
    "E1": (0, 500000),
    "E2": (500000, 750000),
    "E3": (750001, 1000000),
    "D1": (1000001, 1250000),
    "D2": (1250001, 1500000),
    "D3": (1500001, 1750000),
    "C1": (1750001, 2000000),
    "C2": (2000001, 2250000),
    "C3": (2250001, 2500000),
    "B1": (2500001, 2750000),  
    "B2": (2750001, 3000000),  
    "B3": (3000001, 3250000),  
    "A1": (3250001, 3500000),
    "A2": (3500001, 3750000),
    "A3": (3750001, 4000000),
    "S1": (4000001, 6000000),
    "S2": (6000001, 8000000),
    "S3": (8000001, 12000000),
    "L1": (12000001, 14000000),
    "L2": (14000001, 16000000),
    "L3": (16000001, 18000000),
    "L+": (18000001, 22000000),
    "ELITE": (22000001, float('inf'))
}

# Función para asignar la categoría según el precio
def asignar_categoria(precio):
    for categoria, (limite_inferior, limite_superior) in rangos_precio.items():
        if limite_inferior <= precio < limite_superior:
            return categoria
    return None  # En caso de que el precio no caiga en ningún rango (caso raro)

# Asignar la categoría a cada registro
df['categoria'] = df['precio'].apply(asignar_categoria)
print(df['categoria'].unique())

[None 'S1' 'A3' 'A2' 'A1' 'B3' 'B2' 'B1' 'C3' 'C2' 'C1' 'D3' 'D2' 'D1'
 'E3' 'E2' 'E1']


# 2. Meses transcurridos

Sacar el número de meses transcurridos desde la fecha de su publicación a la actual

In [ ]:
from dateutil.relativedelta import relativedelta  # Diferencias entre fechas, incluyendo años, meses y días
from datetime import datetime
# Convertir el mes abreviado en número
def month_to_int(month):
    months = ['ene', 'feb', 'mar', 'abr', 'may', 'jun', 'jul', 'ago', 'sep', 'oct', 'nov', 'dic']
    return months.index(month.lower()) + 1

# Estandarizar el formato de fecha
def standardize_date(date_str):
    # Caso 1: formato "20 abr. 2023" -> "2023-04-20"
    try:
        day, month, year = date_str.split(' ')
        month_int = month_to_int(month.replace('.', ''))  # Eliminar puntos si existen
        return f"{year}-{month_int:02d}-{int(day):02d}"
    except (ValueError, AttributeError):
        pass

    # Caso 2: formato "YYYY-MM-DD HH:MM:SS" o "YYYY-MM-DD"
    try:
        return pd.to_datetime(date_str).strftime('%Y-%m-%d')
    except (ValueError, TypeError):
        # Si no se puede convertir, devolver None
        return None

# Aplicar la función a la columna 'tiempo_de_publicacion'
df['fecha_estandarizada'] = df['tiempo_de_publicacion'].apply(standardize_date)

# Convertir a datetime y manejar errores
df['fecha_estandarizada'] = pd.to_datetime(df['fecha_estandarizada'], errors='coerce')

# Obtener la fecha actual
fecha_actual = datetime.now().date()

# Función para calcular la diferencia en meses
def calcular_meses(fecha):
    if pd.isnull(fecha):
        return None
    rd = relativedelta(fecha_actual, fecha.date())
    return rd.years * 12 + rd.months

# Aplicar la función para calcular los meses transcurridos
df['meses_transcurridos'] = df['fecha_estandarizada'].apply(calcular_meses)

#Registro de fecha de conversión
df['meses_transcurridos_fecha'] = fecha_actual

# Mostrar el DataFrame con las fechas estandarizadas y meses transcurridos
print(df[['tiempo_de_publicacion', 'fecha_estandarizada', 'meses_transcurridos','meses_transcurridos_fecha']])

    tiempo_de_publicacion fecha_estandarizada  meses_transcurridos  \
0              2023-03-29          2023-03-29                   17   
1              2023-04-28          2023-04-28                   16   
2              2024-01-11          2024-01-11                    8   
3              2023-09-13          2023-09-13                   12   
4              2024-02-21          2024-02-21                    6   
..                    ...                 ...                  ...   
127            2024-03-04          2024-03-04                    6   
128            2024-03-04          2024-03-04                    6   
129            2024-02-17          2024-02-17                    7   
130            2023-09-20          2023-09-20                   11   
131            2024-03-04          2024-03-04                    6   

    meses_transcurridos_fecha  
0                  2024-09-19  
1                  2024-09-19  
2                  2024-09-19  
3                  2024-09-19  

# 3. Tipo de propiedad

## a) Lista de tipos deseados

In [ ]:
# Filtrar los registros según los tipos especificados
tipos_permitidos = 'Casa|Casa en Condominio|Departamento|Penthouse|Dúplex|Loft|Estudio|Condominio Horizontal'
df = df[df['tipo'].str.contains(tipos_permitidos, na=False)]
df['tipo'].unique()

array(['Departamento', 'Condominio Horizontal', 'Casa', 'Dúplex',
       'Casa Dúplex', 'Casa en Fraccionamiento', 'Casa en Condominio'],
      dtype=object)

* Casas: Casa en condominio, 
* Departamento: Penthouse, Loft

## b) Manejo de terrenos

Se buscan los registros que no sean las propiedades si no un solo terreno

In [ ]:
# Filtrar los registros según las condiciones dadas
totales =df[df['descripcion'].str.contains('lote', case=False)]
pd.set_option('display.max_colwidth', None) # Mostrar URLs completas

# MANEJAR PROCESO DE ELIMINACIÓN 
for idx, row in totales.iterrows():
    print(f"Registro {idx}:")
    print(row[['url']])#'descripcion',
    
    print("\n¿Te gustaría eliminar el registro?")
    # Solicitar la opción al usuario
    option = input("Elige una opción (si / no): ").strip().lower()
    
    if option == 'si':
        df.drop(idx, inplace=True)
        print(f"Registro eliminado.\n")
    else:
        print(f"Registro conservado.\n")
        print("\n---\n")
print("Proceso completado.")

## C) Estandarizar

In [ ]:
# Convierte todos los valores a strings, maneja NaN o None como cadenas vacías
df['tipo'] = df['tipo'].astype(str)
# Convertir a minúsculas
df['tipo'] = df['tipo'].str.lower()

import unicodedata# Función para eliminar acentos
def eliminar_acentos(texto):
    # Normaliza y elimina los acentos
    texto_normalizado = unicodedata.normalize('NFKD', texto)
    return ''.join(char for char in texto_normalizado if unicodedata.category(char) != 'Mn')
df['tipo'] = df['tipo'].apply(eliminar_acentos)

# Reemplazar Ñ
#df['propiedad'] = df['propiedad'].str.replace('ñ', 'n')
df['tipo'] = df['tipo'].str.replace(r'[^a-zA-Z0-9\s]', '', regex=True)
# Convertir la primera letra de cada palabra a mayúscula
df['tipo'] = df['tipo'].str.title()
df['tipo'].unique()

array(['Casa', 'Casa En Fraccionamiento', 'Departamento',
       'Casa En Condominio', 'Condominio Horizontal'], dtype=object)

# 4. Metros totales

## a) Metros en descripción

In [ ]:
# Función para extraer totales de la descripción
def extraer_totales(descripcion):
    descripcion = descripcion.lower()
    # Buscar patrones de totales
    totales = re.findall(r'\b(\d+[\.,]?\d*)\s*(metros|totales|mt2)\b', descripcion)
    total = None
    for match in totales:
        numero, _ = match
        numero = numero.replace(',', '').replace("'", '')
        try:
            total = float(numero)
        except ValueError:
            continue
        if total:
            break
    return total

# Aplicar la función al DataFrame
df['totales_desc'] = df['descripcion'].apply(extraer_totales)

### Metros [0 / 0]

In [ ]:
# Filtrar los registros según las condiciones dadas
totales = df[( (df['metros_total'] == 0 ) & (df['totales_desc'].isna() | df['totales_desc']==0) )] 
pd.set_option('display.max_colwidth', None) # Mostrar URLs completas

# Iterar sobre los registros filtrados
for idx, row in totales.iterrows():
    print(f"Registro {idx}:")
    print(row[['metros_total', 'totales_desc', 'url']])
    print("\n¿Qué acción te gustaría realizar?")
    print("1. Sustituir el valor manualmente")
    print("2. Sustituir el valor de 'metros_total' por el de 'totales_desc'")
    print("3. Pasar al siguiente registro")

    # Solicitar la opción al usuario
    option = input("Elige una opción (1, 2, 3): ")

    if option == '1':
        # Sustituir el valor manualmente
        new_value = input("Introduce el nuevo valor para 'metros_total': ")
        df.at[idx, 'metros_total'] = float(new_value)
    elif option == '2':
        # Sustituir el valor de 'metros_total' por 'totales_desc'
        df.at[idx, 'metros_total'] = df.at[idx, 'totales_desc']
    elif option == '3':
        # Pasar al siguiente registro
        continue
    else:
        print("Opción no válida. Pasando al siguiente registro...")
    
    print("\n---\n")

print("Proceso completado.")


Proceso completado.


### Metros [0 / #]

In [ ]:
# Filtrar los registros según las condiciones dadas
totales = df[( (df['metros_total'] == 0 ) & (df['totales_desc'].notna()) )] 
pd.set_option('display.max_colwidth', None) # Mostrar URLs completas

# Iterar sobre los registros filtrados
for idx, row in totales.iterrows():
    print(f"Registro {idx}:")
    print(row[['metros_total', 'totales_desc', 'url']])
    print("\n¿Qué acción te gustaría realizar?")
    print("1. Sustituir el valor manualmente")
    print("2. Sustituir el valor de 'metros_total' por el de 'totales_desc'")
    print("3. Pasar al siguiente registro")

    # Solicitar la opción al usuario
    option = input("Elige una opción (1, 2, 3): ")

    if option == '1':
        # Sustituir el valor manualmente
        new_value = input("Introduce el nuevo valor para 'metros_total': ")
        df.at[idx, 'metros_total'] = float(new_value)
    elif option == '2':
        # Sustituir el valor de 'metros_total' por 'totales_desc'
        df.at[idx, 'metros_total'] = df.at[idx, 'totales_desc']
    elif option == '3':
        # Pasar al siguiente registro
        continue
    else:
        print("Opción no válida. Pasando al siguiente registro...")
    
    print("\n---\n")

print("Proceso completado.")


Proceso completado.


### Metros [# / #]

In [ ]:
# Filtrar los registros según las condiciones dadas
totales = df[( ((df['metros_total']!= 0) & (df['totales_desc'].notna())) & (df['metros_total'] != df['totales_desc']) )] 
pd.set_option('display.max_colwidth', None) # Mostrar URLs completas

# Iterar sobre los registros filtrados
for idx, row in totales.iterrows():
    print(f"Registro {idx}:")
    print(f"metros_total: {row['metros_total']}")
    print(f"totales_desc: {row['totales_desc']}")
    print(f"url: {row['url']}")
    
    print("\n¿Qué acción te gustaría realizar?")
    print("1. Sustituir el valor manualmente")
    print("2. Sustituir el valor de 'metros_total' por el de 'totales_desc'")
    print("3. Pasar al siguiente registro")

    # Solicitar la opción al usuario
    option = input("Elige una opción (1, 2, 3): ")

    if option == '1':
        # Sustituir el valor manualmente
        while True:
            try:
                new_value = float(input("Introduce el nuevo valor para 'metros_total': "))
                df.at[idx, 'metros_total'] = new_value
                break
            except ValueError:
                print("Por favor, introduce un número válido (entero o decimal).")
    elif option == '2':
        # Sustituir el valor de 'metros_total' por 'totales_desc'
        df.at[idx, 'metros_total'] = df.at[idx, 'totales_desc']
    elif option == '3':
        # Pasar al siguiente registro
        continue
    else:
        print("Opción no válida. Pasando al siguiente registro...")
    print("\n---\n")

print("Proceso completado.")

## b) Manejo por rangos de UMA

Paso opcional por si no se espera cambiar el precio por el que se tenga en descripción

Rangos
* 118 UMAS = 40m2
* 118.1-200 = 50m2
* 200.1-350 = 71m2
* 350.1-750 = 102m2
* 750.1-1500 = 156m2

### DF cuando ambas columnas son 0 o vacíos

In [ ]:
# Rangos de UMAs y sus equivalencias en m²
umas_to_m2 = [
    (118, 40),
    (200, 50),
    (350, 71),
    (750, 102),
    (1500, 156)
]
valor_uma = 108.57

# Convertir MXN a UMAs
def precio_a_umas(precio, valor_uma):
    return precio / valor_uma

# Asignar m² basado en el precio en UMAs
def asignar_m2(precio_umas):
    for upper_bound, m2 in umas_to_m2:
        if precio_umas <= upper_bound:
            return m2
    return None  # En caso de que el precio en UMAs sea mayor al máximo definido

# Actualizar los registros con 'metros_total' igual a 0 y 'totales_desc' es NaN
def actualizar_metros_total(row):
    if row['metros_total'] == 0 and pd.isna(row['totales_desc']):
        precio_umas = precio_a_umas(row['precio'], valor_uma)
        row['metros_total'] = asignar_m2(precio_umas)
    return row

totales = df[( (df['metros_total'] == 0 ) & (df['totales_desc'].isna() | df['totales_desc']==0) )] 
totales = df.apply(actualizar_metros_total, axis=1)
pd.set_option('display.max_colwidth', None) # Mostrar URLs completas

#df = df.apply(actualizar_metros_total, axis=1)
print(totales[['metros_total', 'totales_desc', 'url']])

In [ ]:
# Si se aceptan los cambios:
df = df.apply(actualizar_metros_total, axis=1)

# 5. M2 construidos

## a) M-construidos en descripción

In [ ]:
# Función para extraer totales de la descripción
def extraer_totales(descripcion):
    descripcion = descripcion.lower()
    # Buscar patrones de totales
    totales = re.findall(r'\b(\d+[\.,]?\d*)\s*(metros|construidos)\b', descripcion)
    total = None
    for match in totales:
        numero, _ = match
        numero = numero.replace(',', '').replace("'", '')
        try:
            total = float(numero)
        except ValueError:
            continue
        if total:
            break
    return total

# Aplicar la función al DataFrame
df['construidos_desc'] = df['descripcion'].apply(extraer_totales)

### Metros [0 / 0]

In [ ]:
# Filtrar los registros según las condiciones dadas
totales = df[( (df['metros_construido'] == 0 ) & (df['construidos_desc'].isna() | df['construidos_desc']==0) )] 
pd.set_option('display.max_colwidth', None) # Mostrar URLs completas

# Iterar sobre los registros filtrados
for idx, row in totales.iterrows():
    print(f"Registro {idx}:")
    print(row[['metros_construido', 'construidos_desc', 'url']])
    print("\n¿Qué acción te gustaría realizar?")
    print("1. Sustituir el valor manualmente")
    print("2. Sustituir el valor de 'metros_construido' por el de 'construidos_desc'")
    print("3. Pasar al siguiente registro")

    # Solicitar la opción al usuario
    option = input("Elige una opción (1, 2, 3): ")

    if option == '1':
        # Sustituir el valor manualmente
        new_value = input("Introduce el nuevo valor para 'metros_construido': ")
        df.at[idx, 'metros_construido'] = float(new_value)
    elif option == '2':
        # Sustituir el valor de 'metros_construido' por 'construidos_desc'
        df.at[idx, 'metros_construido'] = df.at[idx, 'construidos_desc']
    elif option == '3':
        # Pasar al siguiente registro
        continue
    else:
        print("Opción no válida. Pasando al siguiente registro...")
    
    print("\n---\n")

print("Proceso completado.")


Proceso completado.


### Metros [0 / #]

In [ ]:
# Filtrar los registros según las condiciones dadas
totales = df[( (df['metros_construido'] == 0 ) & (df['construidos_desc'].notna()) )] 
pd.set_option('display.max_colwidth', None) # Mostrar URLs completas

# Iterar sobre los registros filtrados
for idx, row in totales.iterrows():
    print(f"Registro {idx}:")
    print(row[['metros_construido', 'construidos_desc', 'url']])
    print("\n¿Qué acción te gustaría realizar?")
    print("1. Sustituir el valor manualmente")
    print("2. Sustituir el valor de 'metros_construido' por el de 'construidos_desc'")
    print("3. Pasar al siguiente registro")

    # Solicitar la opción al usuario
    option = input("Elige una opción (1, 2, 3): ")

    if option == '1':
        # Sustituir el valor manualmente
        new_value = input("Introduce el nuevo valor para 'metros_construido': ")
        df.at[idx, 'metros_construido'] = float(new_value)
    elif option == '2':
        # Sustituir el valor de 'metros_construido' por 'construidos_desc'
        df.at[idx, 'metros_construido'] = df.at[idx, 'construidos_desc']
    elif option == '3':
        # Pasar al siguiente registro
        continue
    else:
        print("Opción no válida. Pasando al siguiente registro...")
    
    print("\n---\n")

print("Proceso completado.")


Proceso completado.


### Metros [# / #]

In [ ]:
# Filtrar los registros según las condiciones dadas
totales = df[( ((df['metros_construido']!= 0) & (df['construidos_desc'].notna())) & (df['metros_construido'] != df['construidos_desc']) )] 
pd.set_option('display.max_colwidth', None) # Mostrar URLs completas

# Iterar sobre los registros filtrados
for idx, row in totales.iterrows():
    print(f"Registro {idx}:")
    print(row[['metros_construido', 'construidos_desc', 'url']])
    print("\n¿Qué acción te gustaría realizar?")
    print("1. Sustituir el valor manualmente")
    print("2. Sustituir el valor de 'metros_construido' por el de 'construidos_desc'")
    print("3. Pasar al siguiente registro")

    # Solicitar la opción al usuario
    option = input("Elige una opción (1, 2, 3): ")

    if option == '1':
        # Sustituir el valor manualmente
        new_value = input("Introduce el nuevo valor para 'metros_construido': ")
        try:
            df.at[idx, 'metros_construido'] = float(new_value)
        except ValueError:
            print("Valor no válido. Debe ser un número. Pasando al siguiente registro...")
    elif option == '2':
        # Sustituir el valor de 'metros_construido' por 'construidos_desc'
        df.at[idx, 'metros_construido'] = df.at[idx, 'construidos_desc']
    elif option == '3':
        # Pasar al siguiente registro
        continue
    else:
        print("Opción no válida. Pasando al siguiente registro...")
    
    print("\n---\n")

print("Proceso completado.")


# 6. Precio / m2

## M2 Totales (terreno)

In [ ]:
# Calcular precio por metro cuadrado del terreno
df['precio_m2_terreno'] = df['precio']/df['metros_total']

## M2 Construidos

In [ ]:
# Calcular precio por metro cuadrado de la cosntrucción
df['precio_m2_construido'] = df['precio']/df['metros_construido']

# 7. Estacionamientos

## a) Estacionamientos descritos

Asigna las diferentes maneras en las que puede encontrarse descrito el espacio de estacionamientos

In [ ]:
# Función para extraer estacionamientos de la descripción
def extraer_coches(descripcion):
    descripcion = descripcion.lower()
    # Buscar patrones de totales
    totales = re.findall(r'\b(\d+[\.,]?\d*)\s*(|cochera|garage|carros|carro|coche|coches|autos|automóviles|automóvil|auto|lugares de estacionamiento)\b', descripcion)
    total = None
    for match in totales:
        numero, _ = match
        numero = numero.replace(',', '').replace("'", '').replace('.','')
        try:
            total = float(numero)
        except ValueError:
            continue
        if total:
            break
    return total

# Aplicar la función al DataFrame
df['estacionamiento_desc'] = df['descripcion'].apply(extraer_coches)

### Estacionamientos [0 / 0]

In [ ]:
# Filtrar los registros según las condiciones dadas
totales = df[( (df['estacionamientos']==0) & (df['estacionamiento_desc'].isna()) )] 
pd.set_option('display.max_colwidth', None) # Mostrar URLs completas

# Iterar sobre los registros filtrados
for idx, row in totales.iterrows():
    print(f"Registro {idx}:")
    print(row[['url']])
    print("\n¿Qué acción te gustaría realizar?")
    print("1. Sustituir el valor manualmente")
    print("2. Pasar al siguiente registro")

    # Solicitar la opción al usuario
    option = input("Elige una opción (1 | 2): ")

    if option == '1':
        # Sustituir el valor manualmente
        new_value = input("Introduce el nuevo valor para 'estacionamientos': ")
        df.at[idx, 'estacionamientos'] = float(new_value)    
    elif option == '2':
        # Pasar al siguiente registro
        continue
    else:
        print("Opción no válida. Pasando al siguiente registro...")
    
    print("\n---\n")

print("Proceso completado.")

Proceso completado.


### Estacionamientos [0 / #]

In [ ]:
# Filtrar los registros según las condiciones dadas
totales = df[( (df['estacionamientos']==0) & (df['estacionamiento_desc'].notna()) )] 
pd.set_option('display.max_colwidth', None) # Mostrar URLs completas

# Iterar sobre los registros filtrados
for idx, row in totales.iterrows():
    print(f"Registro {idx}:")
    print(row[['estacionamientos', 'estacionamiento_desc', 'url']])
    print("\n¿Qué acción te gustaría realizar?")
    print("1. Sustituir el valor manualmente")
    print("2. Sustituir el valor de 'estacionamientos' por el de 'estacionamiento_desc'")
    print("3. Pasar al siguiente registro")

    # Solicitar la opción al usuario
    option = input("Elige una opción (1, 2, 3): ")

    if option == '1':
        # Sustituir el valor manualmente
        new_value = input("Introduce el nuevo valor para 'estacionamientos': ")
        df.at[idx, 'estacionamientos'] = float(new_value)
    elif option == '2':
        # Sustituir el valor de 'metros_total' por 'totales_desc'
        df.at[idx, 'estacionamientos'] = df.at[idx, 'estacionamiento_desc']
    elif option == '3':
        # Pasar al siguiente registro
        continue
    else:
        print("Opción no válida. Pasando al siguiente registro...")
    
    print("\n---\n")

print("Proceso completado.")

Proceso completado.


### Estacionamientos [# / #]

In [ ]:
# Filtrar los registros según las condiciones dadas
totales = df[( ((df['estacionamientos']!=0) & (df['estacionamiento_desc'].notna())) & (df['estacionamientos'] != df['estacionamiento_desc']) )] 
pd.set_option('display.max_colwidth', None) # Mostrar URLs completas

# Iterar sobre los registros filtrados
for idx, row in totales.iterrows():
    print(f"Registro {idx}:")
    print(row[['estacionamientos', 'estacionamiento_desc', 'url']])
    print("\n¿Qué acción te gustaría realizar?")
    print("1. Sustituir el valor manualmente")
    print("2. Sustituir el valor de 'estacionamientos' por el de 'estacionamiento_desc'")
    print("3. Pasar al siguiente registro")

    # Solicitar la opción al usuario
    option = input("Elige una opción (1, 2, 3): ")

    if option == '1':
        # Sustituir el valor manualmente
        new_value = input("Introduce el nuevo valor para 'estacionamientos': ")
        df.at[idx, 'estacionamientos'] = float(new_value)
    elif option == '2':
        # Sustituir el valor de 'metros_total' por 'totales_desc'
        df.at[idx, 'estacionamientos'] = df.at[idx, 'estacionamiento_desc']
    elif option == '3':
        # Pasar al siguiente registro
        continue
    else:
        print("Opción no válida. Pasando al siguiente registro...")
    
    print("\n---\n")

print("Proceso completado.")

## b) Mantener rango de estacionamientos < 5 (mínimo 1)

In [ ]:
df = df[df['estacionamientos'] <= 8]
#df[['estacionamientos','estacionamiento_desc','url']]

# 8. Recámaras

## a) Recámaras en descripción

In [ ]:
# Función para extraer cantidad de recamaras en descripción
def extraer_recamaras(descripcion):
    descripcion = descripcion.lower()
    # Buscar patrones de totales
    totales = re.findall(r'\b(\d+[\.,]?\d*)\s*(recamara|recamaras|habitaciones|dormitorio|dormitorios)\b', descripcion)
    total = None
    for match in totales:
        numero, _ = match
        numero = numero.replace(',', '').replace("'", '')
        try:
            total = float(numero)
        except ValueError:
            continue
        if total:
            break
    return total

# Aplicar la función al DataFrame
df['recamaras_desc'] = df['descripcion'].apply(extraer_recamaras)

### Recámaras [0 / 0]

In [ ]:
# Filtrar los registros según las condiciones dadas
totales = df[ (df['recamaras'] == 0 ) & (df['recamaras_desc'].isna()) ] 
pd.set_option('display.max_colwidth', None) # Mostrar URLs completas

# Iterar sobre los registros filtrados
for idx, row in totales.iterrows():
    print(f"Registro {idx}:")
    print(row[['url']])
    print("\n¿Qué acción te gustaría realizar?")
    print("1. Sustituir el valor manualmente")
    print("2. Pasar al siguiente registro")

    # Solicitar la opción al usuario
    option = input("Elige una opción (1 | 2): ")

    if option == '1':
        # Sustituir el valor manualmente
        new_value = input("Introduce el nuevo valor para 'recamaras': ")
        df.at[idx, 'recamaras'] = float(new_value)
    elif option == '2':
        # Pasar al siguiente registro
        continue
    else:
        print("Opción no válida. Pasando al siguiente registro...")
    
    print("\n---\n")

print("Proceso completado.")

Proceso completado.


### Recámaras [0 / #]

In [ ]:
# Filtrar los registros según las condiciones dadas
totales = df[( (df['recamaras']==0) & (df['recamaras_desc'].notna()) )] 
pd.set_option('display.max_colwidth', None) # Mostrar URLs completas

# Iterar sobre los registros filtrados
for idx, row in totales.iterrows():
    print(f"Registro {idx}:")
    print(row[['recamaras', 'recamaras_desc', 'url']])
    print("\n¿Qué acción te gustaría realizar?")
    print("1. Sustituir el valor manualmente")
    print("2. Sustituir el valor de 'recamaras' por el de 'recamaras_desc'")
    print("3. Pasar al siguiente registro")

    # Solicitar la opción al usuario
    option = input("Elige una opción (1, 2, 3): ")

    if option == '1':
        # Sustituir el valor manualmente
        new_value = input("Introduce el nuevo valor para 'recamaras': ")
        df.at[idx, 'recamaras'] = float(new_value)
    elif option == '2':
        # Sustituir el valor de 'metros_total' por 'totales_desc'
        df.at[idx, 'recamaras'] = df.at[idx, 'recamaras_desc']
    elif option == '3':
        # Pasar al siguiente registro
        continue
    else:
        print("Opción no válida. Pasando al siguiente registro...")
    
    print("\n---\n")

print("Proceso completado.")


Proceso completado.


### Recámaras [# / #]

In [ ]:
# Filtrar los registros según las condiciones dadas
totales = df[( (df['recamaras']!=0) & (df['recamaras_desc'].notna()) ) & (df['recamaras'] != df['recamaras_desc'])] 
pd.set_option('display.max_colwidth', None) # Mostrar URLs completas

# Iterar sobre los registros filtrados
for idx, row in totales.iterrows():
    print(f"Registro {idx}:")
    print(row[['recamaras', 'recamaras_desc', 'url']])
    print("\n¿Qué acción te gustaría realizar?")
    print("1. Sustituir el valor manualmente")
    print("2. Sustituir el valor de 'recamaras' por el de 'recamaras_desc'")
    print("3. Pasar al siguiente registro")

    # Solicitar la opción al usuario
    option = input("Elige una opción (1, 2, 3): ")

    if option == '1':
        # Sustituir el valor manualmente
        new_value = input("Introduce el nuevo valor para 'recamaras': ")
        df.at[idx, 'recamaras'] = float(new_value)
    elif option == '2':
        # Sustituir el valor de 'metros_total' por 'totales_desc'
        df.at[idx, 'recamaras'] = df.at[idx, 'recamaras_desc']
    elif option == '3':
        # Pasar al siguiente registro
        continue
    else:
        print("Opción no válida. Pasando al siguiente registro...")
    
    print("\n---\n")

print("Proceso completado.")


## b) Mantener rango de recámaras <= 7 

In [ ]:
print(df['recamaras'].unique())
df = df[(df['recamaras'] <= 7)] 
#print(df[['recamaras','recamaras_desc','url']])

# 9. Baños

## a) Baños en descripción

Manejando tanto los completos como los medios

In [ ]:
# Extraer la cantidad de baños y medios baños en la descripción
def extraer_baños(descripcion):
    descripcion = descripcion.lower()
    
    # Buscar patrones de baños completos
    completos = re.findall(r'\b(\d+[\.,]?\d*)\s*baños?\b', descripcion)
    total_completos = sum([float(match.replace(',', '').replace("'", '')) for match in completos])
    
    # Buscar patrones de medios baños (1/2 baño)
    medios = re.findall(r'\b(\d+[\.,]?\d*)\s*medios?\s*baños?\b', descripcion)
    total_medios = sum([float(match.replace(',', '').replace("'", '')) for match in medios])
    
    # Retornar como un diccionario o una tupla
    return {'completos': total_completos, 'medios': total_medios}

# Aplicar la función al DataFrame
df['baños_desc'] = df['descripcion'].apply(extraer_baños)

# Si deseas separar en dos columnas distintas:
df['baños_completos'] = df['baños_desc'].apply(lambda x: x['completos'] if x else 0)
df['medios_baños'] = df['baños_desc'].apply(lambda x: x['medios'] if x else 0)

### Baños

#### [0 / 0]

Poner como mínimo para el registro 1 en baños, pero si cuenta con medio baño en descripción que se le asigne ese valor.

In [ ]:
# Filtrar los registros según las condiciones dadas
totales = df[(df['banos'] == 0) & (df['baños_completos'].isna())]
pd.set_option('display.max_colwidth', None) # Mostrar URLs completas

# Iterar sobre los registros filtrados
for idx, row in totales.iterrows():
    print(f"Registro {idx}:")
    print(row[['url']])
    print("\n¿Qué acción te gustaría realizar?")
    print("1. Sustituir el valor manualmente")
    #print("2. Asignar 1 como mínimo y pasar al siguiente registro")
    print("2. Pasar al siguiente registro")

    # Solicitar la opción al usuario
    option = input("Elige una opción (1 | 2): ")

    if option == '1':
        # Sustituir el valor manualmente
        new_value = input("Introduce el nuevo valor para 'banos': ")
        df.at[idx, 'banos'] = float(new_value)
    #elif option == '2':
        # Asignar valor de 1 como mínimo
        #df.at[idx, 'banos'] = 1
        #continue  # Pasar al siguiente registro
    else:
        continue  # Pasar al siguiente registro

    print("\n---\n")

print("Proceso completado.")

Proceso completado.


#### [0 / #]

In [ ]:
# Filtrar los registros según las condiciones dadas
totales = df[( (df['banos']==0) & (df['baños_completos'].notna()) )] 
pd.set_option('display.max_colwidth', None) # Mostrar URLs completas

# Iterar sobre los registros filtrados
for idx, row in totales.iterrows():
    print(f"Registro {idx}:")
    print(row[['banos']])
    print(row[['baños_completos']])
    print(row[['url']])
    print("\n¿Qué acción te gustaría realizar?")
    print("1. Sustituir el valor manualmente")
    print("2. Sustituir el valor de 'banos' por el de 'baños_completos'")
    #print("3. Asignar 1 como mínimo y pasar al siguiente registro")
    print("3. Pasar al siguiente registro")

    # Solicitar la opción al usuario
    option = input("Elige una opción (1 | 2 | 3): ")

    if option == '1':
        # Sustituir el valor manualmente
        new_value = input("Introduce el nuevo valor para 'banos': ")
        df.at[idx, 'banos'] = float(new_value)
    elif option == '2':
        # Sustituir el valor de 'banos' por 'baños_completos'
        df.at[idx, 'banos'] = df.at[idx, 'baños_completos']
    #elif option == '3':
        # Asignar valor de 1 como mínimo
        #df.at[idx, 'banos'] = 1
        #continue  # Pasar al siguiente registro
    else:
        continue  # Pasar al siguiente registro

    print("\n---\n")

print("Proceso completado.")

Registro 236:
banos    0.0
Name: 236, dtype: object
baños_completos    0.0
Name: 236, dtype: object
url    https://www.lamudi.com.mx/detalle/41032-73-a3ee8bf8fc9d-524a-1f3db509-83cd-4b7b
Name: 236, dtype: object

¿Qué acción te gustaría realizar?
1. Sustituir el valor manualmente
2. Sustituir el valor de 'banos' por el de 'baños_completos'
3. Pasar al siguiente registro


KeyboardInterrupt: Interrupted by user

#### [# / #]

In [ ]:
# Filtrar los registros según las condiciones dadas
totales = df[( ((df['banos']!=0) & (df['baños_completos'].notna())) & (df['banos'] != df['baños_completos']))] 
pd.set_option('display.max_colwidth', None) # Mostrar URLs completas

# Iterar sobre los registros filtrados
for idx, row in totales.iterrows():
    print(f"Registro {idx}:")
    print(row[['banos']])
    print(row[['baños_completos']])    
    print(row[['url']])
    
    print("\n¿Qué acción te gustaría realizar?")
    print("1. Sustituir el valor manualmente")
    print("2. Sustituir el valor de 'banos' por el de 'baños_completos'")
    #print("3. Asignar 1 como mínimo y pasar al siguiente registro")
    print("3. Pasar al siguiente registro")

    # Solicitar la opción al usuario
    option = input("Elige una opción (1 | 2 | 3): ")

    if option == '1':
        # Sustituir el valor manualmente
        new_value = input("Introduce el nuevo valor para 'banos': ")
        df.at[idx, 'banos'] = float(new_value)
    elif option == '2':
        # Sustituir el valor de 'banos' por 'baños_completos'
        df.at[idx, 'banos'] = df.at[idx, 'baños_completos']
    #elif option == '3':
        # Asignar valor de 1 como mínimo
        #df.at[idx, 'banos'] = 1
        #continue  # Pasar al siguiente registro
    else:
        continue  # Pasar al siguiente registro

    print("\n---\n")

print("Proceso completado.")

Registro 31:
banos    2
Name: 31, dtype: object
baños_completos    5.0
Name: 31, dtype: object
url    https://www.lamudi.com.mx/detalle/41032-73-93d194dc33d-d035-bf07501c-825e-4a4a
Name: 31, dtype: object

¿Qué acción te gustaría realizar?
1. Sustituir el valor manualmente
2. Sustituir el valor de 'banos' por el de 'baños_completos'
3. Pasar al siguiente registro
Registro 37:
banos    3
Name: 37, dtype: object
baños_completos    0.0
Name: 37, dtype: object
url    https://www.lamudi.com.mx/detalle/41032-73-27915b7692e9-3bc0-67bbfd01-9453-4608
Name: 37, dtype: object

¿Qué acción te gustaría realizar?
1. Sustituir el valor manualmente
2. Sustituir el valor de 'banos' por el de 'baños_completos'
3. Pasar al siguiente registro
Registro 65:
banos    1
Name: 65, dtype: object
baños_completos    0.0
Name: 65, dtype: object
url    https://www.lamudi.com.mx/detalle/41032-73-c7136c4c8f36-cb2f-fd601220-8b1e-4305
Name: 65, dtype: object

¿Qué acción te gustaría realizar?
1. Sustituir el valor manu

C:\Users\yoe11\AppData\Local\Temp\ipykernel_10972\1369790780.py:26: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '3.5' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.at[idx, 'banos'] = df.at[idx, 'baños_completos']



---

Registro 6993:
banos    1
Name: 6993, dtype: object
baños_completos    0.0
Name: 6993, dtype: object
url    https://www.lamudi.com.mx/detalle/41032-73-eb0c79af90fc-430-e6bbff06-a0f7-33ad
Name: 6993, dtype: object

¿Qué acción te gustaría realizar?
1. Sustituir el valor manualmente
2. Sustituir el valor de 'banos' por el de 'baños_completos'
3. Pasar al siguiente registro
Registro 7019:
banos    1
Name: 7019, dtype: object
baños_completos    0.0
Name: 7019, dtype: object
url    https://www.lamudi.com.mx/detalle/41032-73-50efc0ce8b46-81d9-289506e0-957b-402a
Name: 7019, dtype: object

¿Qué acción te gustaría realizar?
1. Sustituir el valor manualmente
2. Sustituir el valor de 'banos' por el de 'baños_completos'
3. Pasar al siguiente registro
Registro 7020:
banos    1
Name: 7020, dtype: object
baños_completos    0.0
Name: 7020, dtype: object
url    https://www.lamudi.com.mx/detalle/41032-73-b89ad56639c5-4da8-7f8d22f8-99a6-3892
Name: 7020, dtype: object

¿Qué acción te gustaría realiz

### Medio baños

#### [0 / 0]

Poner como mínimo para el registro 1 en baños, pero si cuenta con valor en descripción que se le asigne ese valor.

In [ ]:
# Filtrar los registros según las condiciones dadas
totales = df[(df['medio_baños'] == 0) & (df['medios_baños'].isna())]
pd.set_option('display.max_colwidth', None) # Mostrar URLs completas

# Iterar sobre los registros filtrados
for idx, row in totales.iterrows():
    print(f"Registro {idx}:")
    print(row[['url']])
    print("\n¿Qué acción te gustaría realizar?")
    print("1. Sustituir el valor manualmente")
    #print("2. Asignar 1 como mínimo y pasar al siguiente registro")
    print("2. Pasar al siguiente registro")

    # Solicitar la opción al usuario
    option = input("Elige una opción (1 | 2): ")

    if option == '1':
        # Sustituir el valor manualmente
        new_value = input("Introduce el nuevo valor para 'banos': ")
        df.at[idx, 'banos'] = float(new_value)
    #elif option == '2':
        # Asignar valor de 1 como mínimo
        #df.at[idx, 'banos'] = 1
        #continue  # Pasar al siguiente registro
    else:
        continue  # Pasar al siguiente registro

    print("\n---\n")

print("Proceso completado.")

KeyError: 'medio_baños'

#### [0 / #]

In [ ]:
# Filtrar los registros según las condiciones dadas
totales = df[( (df['medio_baños']==0) & (df['medios_baños'].notna()) )] 
pd.set_option('display.max_colwidth', None) # Mostrar URLs completas

# Iterar sobre los registros filtrados
for idx, row in totales.iterrows():
    print(f"Registro {idx}:")
    print(row[['url']])
    print("\n¿Qué acción te gustaría realizar?")
    print("1. Sustituir el valor manualmente")
    print("2. Sustituir el valor de 'medio_baños' por el de 'medios_baños'")
    #print("3. Asignar 1 como mínimo y pasar al siguiente registro")
    print("3. Pasar al siguiente registro")

    # Solicitar la opción al usuario
    option = input("Elige una opción (1 | 2 | 3): ")

    if option == '1':
        # Sustituir el valor manualmente
        new_value = input("Introduce el nuevo valor para 'medio_baños': ")
        df.at[idx, 'medio_baños'] = float(new_value)
    elif option == '2':
        # Sustituir el valor de 'medio_baños' por 'medios_baños'
        df.at[idx, 'medio_baños'] = df.at[idx, 'medios_baños']
    #elif option == '3':
        # Asignar valor de 1 como mínimo
        #df.at[idx, 'medio_baños'] = 1
        #continue  # Pasar al siguiente registro
    else:
        continue  # Pasar al siguiente registro

    print("\n---\n")

print("Proceso completado.")

#### [# / #]

In [ ]:
# Filtrar los registros según las condiciones dadas
totales = df[( ((df['medio_baños']!=0) & (df['medios_baños'].notna())) & (df['medio_baños'] != df['medios_baños']))] 
pd.set_option('display.max_colwidth', None) # Mostrar URLs completas

# Iterar sobre los registros filtrados
for idx, row in totales.iterrows():
    print(f"Registro {idx}:")
    print(row[['url']])
    print("\n¿Qué acción te gustaría realizar?")
    print("1. Sustituir el valor manualmente")
    print("2. Sustituir el valor de 'medio_baños' por el de 'medios_baños'")
    #print("3. Asignar 1 como mínimo y pasar al siguiente registro")
    print("3. Pasar al siguiente registro")

    # Solicitar la opción al usuario
    option = input("Elige una opción (1 | 2 | 3): ")

    if option == '1':
        # Sustituir el valor manualmente
        new_value = input("Introduce el nuevo valor para 'medio_baños': ")
        df.at[idx, 'medio_baños'] = float(new_value)
    elif option == '2':
        # Sustituir el valor de 'medio_baños' por 'medios_baños'
        df.at[idx, 'medio_baños'] = df.at[idx, 'medios_baños']
    #elif option == '3':
        # Asignar valor de 1 como mínimo
        #df.at[idx, 'medio_baños'] = 1
        #continue  # Pasar al siguiente registro
    else:
        continue  # Pasar al siguiente registro

    print("\n---\n")

print("Proceso completado.")

## B) Total de baños

In [ ]:
# Make the medio_baños and banos sum
# 'medio_baños', divididos entre 2 para representar medios
df['bano_total'] = df['banos'] + (df['medio_banos'] / 2)
df['bano_total'] = df['bano_total'].round(1)  # Round the total, not just 'banos'
print(df[['banos', 'medio_banos', 'bano_total']].head())

   banos  medio_banos  bano_total
0      5            0         5.0
1      0            0         0.0
2      0            0         0.0
3      1            0         1.0
4      1            0         1.0
